<a href="https://colab.research.google.com/github/aminkakartinkadotpng/AMINA-S-INNER-WORLD/blob/main/Lab_2_Jexembayeva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа № 2
# Джексембаева Амина Андреевна
# Группа: М10-415БКИ-19

Подключаем библиотеки


In [17]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

Загружаем датасет (breast_cancer), заменяем буквы на цифры с помощью labelEncoder- датафрейм
Делаем из датафрема 2 массива: features и labels



In [29]:
df = pd.read_csv('/content/sample_data/breast_cancer.csv')
labelencoder_df = LabelEncoder() 
labels = labelencoder_df.fit_transform(df.diagnosis)
features = ['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean','radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se','radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst']
x = df[features]
features = np.array(x)

# Делим данные на тестовую и тренировочную выборку


In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

Создаем 3 дополнительных функций для обучения: fit_model - функция которая будет обучать модели; write_model- функция которая записывает данные и параметры модели в файлик; result_model - функция, которая выписывает метрики модели


In [39]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score,roc_auc_score
def fit_model(name, model, x_train, y_train):
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  result_model(y_pred)
  write_model(model, name)

def write_model(model, name):
  model_pkl = open(f"/content/sample_data/{name}.pkl", "wb")
  pickle.dump(model, model_pkl, protocol=3)
  model_pkl.close()
  model_params = open(f"/content/sample_data/{name}_params.txt", "w")
  model_params.write(str(model.best_params_))
  model_params.close()
  print(model.best_params_)

def result_model(y_pred):
  print(
  f'Confusion matrix:\n {confusion_matrix(y_test, y_pred)}\n\n'
  f'Accuracy:\t\t{accuracy_score(y_test, y_pred)}\n'
  f'Recall:\t\t\t{recall_score(y_test, y_pred)}\n'
  f'Precision:\t\t{precision_score(y_test, y_pred)}\n'
  f'ROC_AUC curve:\t\t{roc_auc_score(y_test, y_pred)}\n')

# 1. Метод опорных векторов
Основная идея метода заключается в построении гиперплоскости, разделяющей объекты выборки оптимальным способом. Алгоритм работает в предположении, что чем больше расстояние (зазор) между разделяющей гиперплоскостью и объектами разделяемых классов, тем меньше будет средняя ошибка классификатора. 

In [40]:
svc_m_pipline = Pipeline([('preprocess', StandardScaler()), ('svcm', SVC())])
parameters = [
    {'svcm__C': [1,13], 'svcm__kernel': ['linear']},
    {'svcm__C': [1,13], 'svcm__kernel': ['poly'], 'svcm__degree': [1,6], 'svcm__gamma': ['scale', 'auto'], 'svcm__coef0': [-1,1]},
    {'svcm__C': [1,13], 'svcm__kernel': ['rbf'], 'svcm__degree': [1,6], 'svcm__gamma': ['scale', 'auto']},
    {'svcm__C': [1,13], 'svcm__kernel': ['sigmoid'], 'svcm__degree': [1,6], 'svcm__gamma': ['scale', 'auto'], 'svcm__coef0': [-1,1]}]
svc_model = GridSearchCV(svc_m_pipline, parameters, n_jobs=-1)
fit_model('svc', svc_model, x_train, y_train)

Confusion matrix:
 [[69  1]
 [ 2 42]]

Accuracy:		0.9736842105263158
Recall:			0.9545454545454546
Precision:		0.9767441860465116
ROC_AUC curve:		0.9701298701298702

{'svcm__C': 1, 'svcm__degree': 1, 'svcm__gamma': 'scale', 'svcm__kernel': 'rbf'}


# 2. Метод ближайших соседей
Близкие друг к другу наблюдения называются “соседи”. Когда представляется новое наблюдение, обозначенное знаком вопроса, вычисляется его расстояние от всех других наблюдений в модели. Определяется классификация наиболее похожих наблюдений (ближайшее сходство) и новое наблюдение помещается в категорию, в которой содержится наибольшее количество ближайшего сходства. 

In [41]:
knn_pipeline = Pipeline([('preprocess', StandardScaler()), ('knn', KNeighborsClassifier())])
parameters = [
    {'knn__n_neighbors': [1, 15], 'knn__weights': ['uniform', 'distance'], 'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'knn__leaf_size': [5, 70], 'knn__p': [1,2]},
    {'knn__n_neighbors': [1, 15], 'knn__weights': ['uniform', 'distance'], 'knn__algorithm': ['auto','brute'], 'knn__p': [1,2]}]
knn_model = GridSearchCV(knn_pipeline, parameters)
fit_model('knn', knn_model, x_train, y_train)

Confusion matrix:
 [[70  0]
 [ 4 40]]

Accuracy:		0.9649122807017544
Recall:			0.9090909090909091
Precision:		1.0
ROC_AUC curve:		0.9545454545454546

{'knn__algorithm': 'auto', 'knn__leaf_size': 5, 'knn__n_neighbors': 15, 'knn__p': 2, 'knn__weights': 'distance'}


# 3. Наивный метод Байеса
Основана на теореме Байеса: Теорема Байеса — это простая математическая формула, используемая для вычисления условных вероятностей.


In [42]:
bay_pipeline = Pipeline([('preprocess', StandardScaler()),('bay', GaussianNB())])
parameters = [
    {'bay__var_smoothing': [1e-10, 1.0]}]
bay_model = GridSearchCV(bay_pipeline, parameters)
fit_model('bay', bay_model, x_train, y_train )

Confusion matrix:
 [[69  1]
 [ 5 39]]

Accuracy:		0.9473684210526315
Recall:			0.8863636363636364
Precision:		0.975
ROC_AUC curve:		0.936038961038961

{'bay__var_smoothing': 1e-10}


# 4. Метод «случайный лес»
Каждое отдельное дерево в таком лесу дает предсказание класса, и набравший наибольшее количество голосов Класс (Class), становится предсказанием Модели (Model).

In [43]:
r_forest_pipeline = Pipeline([('preprocess', StandardScaler()), ('rfr', RandomForestClassifier())])
parameters = [
    {'rfr__n_estimators': [60, 140], 'rfr__criterion': ('gini', 'entropy'), 'rfr__min_samples_split': [2, 5], 
     'rfr__min_samples_leaf': [2, 4], 'rfr__min_weight_fraction_leaf': [0.0, 0.1], 'rfr__max_features': ('sqrt', 'log2'), 
     'rfr__min_impurity_decrease': [0.0, 0.1], 'rfr__ccp_alpha':[0.0, 0.2], 'rfr__min_impurity_decrease': [0.0]}]
r_forest_model = GridSearchCV(r_forest_pipeline, parameters, n_jobs=-1)
fit_model('r_forest', r_forest_model, x_train, y_train)

Confusion matrix:
 [[69  1]
 [ 4 40]]

Accuracy:		0.956140350877193
Recall:			0.9090909090909091
Precision:		0.975609756097561
ROC_AUC curve:		0.9474025974025975

{'rfr__ccp_alpha': 0.0, 'rfr__criterion': 'entropy', 'rfr__max_features': 'sqrt', 'rfr__min_impurity_decrease': 0.0, 'rfr__min_samples_leaf': 2, 'rfr__min_samples_split': 2, 'rfr__min_weight_fraction_leaf': 0.0, 'rfr__n_estimators': 60}


# 5. Метод логической регрессии
Метод для анализа набора данных, в котором есть одна или несколько независимых переменных, которые определяют результат

In [44]:
simplefilter("ignore", category=ConvergenceWarning)
logreg = LogisticRegression(max_iter=250)
logreg_pipline = Pipeline([('preprocess', StandardScaler()), ('LogisticRegression', logreg)])
parameters = [
    {'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['lbfgs'], 'LogisticRegression__C': [1, 10]},
    {'LogisticRegression__penalty': ['l2', 'l1'], 'LogisticRegression__solver': ['liblinear'], 'LogisticRegression__C': [1, 10]},
    {'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['newton-cg'], 'LogisticRegression__C': [1, 10]},
    {'LogisticRegression__penalty': ['l2'], 'LogisticRegression__solver': ['sag'], 'LogisticRegression__C': [1, 10]},
    {'LogisticRegression__penalty': ['l1', 'l2'], 'LogisticRegression__solver': ['saga'], 'LogisticRegression__C': [1, 10]}]
logreg_model = GridSearchCV(logreg_pipline, parameters)
fit_model('LogisticRegression', logreg_model, x_train, y_train)

Confusion matrix:
 [[69  1]
 [ 1 43]]

Accuracy:		0.9824561403508771
Recall:			0.9772727272727273
Precision:		0.9772727272727273
ROC_AUC curve:		0.9814935064935065

{'LogisticRegression__C': 10, 'LogisticRegression__penalty': 'l1', 'LogisticRegression__solver': 'saga'}


In [45]:
result = pd.DataFrame({'model': ['SVC','KNeighborsClassifier', 'GaussianNB', 'RandomForest', 'LogisticRegression'], 
                       'Accuracy': [0.973,  0.964, 0.947, 0.956, 0.982], 
                       'Recall': [ 0.954,  0.909, 0.886,0.909,0.977], 
                       'Precision': [ 0.976,  1.0, 0.975,0.975,0.977], 
                       'ROC_AUC curve': [	0.970,0.954, 0.936,0.947,0.981]})
result

,model,Accuracy,Recall,Precision,ROC_AUC curve
0,SVC,0.973,0.954,0.976,0.970
1,KNeighborsClassifier,0.964,0.909,1.000,0.954
2,GaussianNB,0.947,0.886,0.975,0.936
3,RandomForest,0.956,0.909,0.975,0.947
4,LogisticRegression,0.982,0.977,0.977,0.981


# Вывод:

1.   Наилучшие результаты показал метод линейной регрессии. Точность- 98,2% Полнота- 97,7%

2.  Наихудшие результаты показал наивный метод Байеса. Точность- 94,7% Полнота- 88,6%

Обусловлено это тем, что в первой лабораторной работе у нас была высокая корреляция.
